In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
from __future__ import print_function
import json

from datetime import date
import datetime
#imports

There are 20 teams in the PL, each plays 38 matches for a total of 760. The first game in the 2022-2023 season is number 18202, and it counts up one by one from there, so the total range will be 18,202 to 18,961. Then we will take only the completed games from there.

NEXT: Will add something to get all of the players from the match, and then count the defensive players to determine back 4 or back 5

Code will be -- take all of the elements where the class "player-position" is DC... get a count, if it's two then the team will be in a back four, if there are 3 then it's a back five with wingbacks

In [2]:
# d = range(18202,18962)
match_range = range(18202,18400)

In [3]:
urls = []
for i in match_range:
    base_url = f'https://understat.com/match/{i}'
    urls.append(base_url)

In [4]:
all_res_data = []
for i in urls:
    res = requests.get(i)
    soup = BeautifulSoup(res.content, 'lxml')
    scripts = soup.find_all('script')
#     positions = soup.find_all("span", {"class": "player-position"})

    all_res_data.append(scripts)
    #Here we can append only the ones that have 'script'

In [85]:
backup_all_res_data = all_res_data
#Adding a backup unless we need it and so we don't need to scrape again

In [86]:
all_res_data = backup_all_res_data

In [87]:
scripts = []
for i in all_res_data:
    if len(i) > 1:
        scripts.append(i)
    else:
        pass
#All of the incomplete games in all_res_data will have a length of zero, so this is how we will filter out the unplayed games
#if necessary

In [88]:
script_list = []
for i in scripts:
    script_list.append(i[1])
#The [1] element is the one with all of the shot data

In [89]:
for i in range(0, len(script_list)):
    script_list[i] = str(script_list[i])
#turning all of our data into strings

In [90]:
data_list = []

# strip unnecessary symbols and get only JSON data 
for i in range(0, len(script_list)):
    ind_start = script_list[i].index("('")+2 
    ind_end = script_list[i].index("')") 
    json_data = script_list[i][ind_start:ind_end] 
    json_data = json_data.encode('utf8').decode('unicode_escape')

#convert string to json format
    data = json.loads(json_data)
    data_list.append(data)

In [91]:
all_data = []
match_id_list = []
x_list = []
y_list = []
home_score_list = []
away_score_list = []
player_list = []
xG_list = []
h_a_list = []
situation_list = []
result_list = []
team_list = []
opposing_team_list = []
minute_list = []
length = len(data_list)
for i in range(length):
    match_id = [] 
    x = []
    y = []
    home_score = []
    away_score = []
    player = []
    xG = []
    h_a = []
    situation = []
    result = []
    team = []
    opposing_team = []
    minute = []
    data_away = data_list[i]['a']
    data_home = data_list[i]['h']

    for index in range(len(data_home)):
        for key in data_home[index]:
            if key == 'X':
                x.append(data_home[index][key])
            if key == 'Y':
                y.append(data_home[index][key])
            if key == 'h_team':
                team.append(data_home[index][key])
            if key == 'a_team':
                opposing_team.append(data_home[index][key])
            if key == 'xG':
                xG.append(data_home[index][key])
            if key == 'situation':
                situation.append(data_home[index][key])
            if key == 'result':
                result.append(data_home[index][key])
            if key == 'h_goals':
                home_score.append(data_home[index][key])
            if key == 'a_goals':
                away_score.append(data_home[index][key])
            if key == 'minute':
                minute.append(data_home[index][key]) 
            if key == 'player':
                player.append(data_home[index][key])
            if key == 'match_id':
                match_id.append(data_home[index][key])
            if key == 'h_a':
                h_a.append(data_home[index][key])
        
        #NEED TO APPEND EVERYTHING TO ALL_DATA

    for index in range(len(data_away)):
        for key in data_away[index]:
            if key == 'X':
                x.append(data_away[index][key])
            if key == 'Y':
                y.append(data_away[index][key])
            if key == 'a_team':
                team.append(data_away[index][key])
            if key == 'h_team':
                opposing_team.append(data_away[index][key])
            if key == 'xG':
                xG.append(data_away[index][key])
            if key == 'situation':
                situation.append(data_away[index][key])
            if key == 'result':
                result.append(data_away[index][key])
            if key == 'h_goals':
                home_score.append(data_away[index][key])
            if key == 'a_goals':
                away_score.append(data_away[index][key])
            if key == 'minute':
                minute.append(data_away[index][key])
            if key == 'player':
                player.append(data_away[index][key])
            if key == 'match_id':
                match_id.append(data_away[index][key])
            if key == 'h_a':
                h_a.append(data_away[index][key]) 
                
    match_id_list.append(match_id)
    x_list.append(x)
    y_list.append(y)
    team_list.append(team)
    opposing_team_list.append(opposing_team)
    xG_list.append(xG)
    situation_list.append(situation)
    result_list.append(result)
    home_score_list.append(home_score)
    away_score_list.append(away_score)
    minute_list.append(minute)
    player_list.append(player)
    h_a_list.append(h_a)

In [92]:
match_id_list = [val for sublist in match_id_list for val in sublist]
x_list = [val for sublist in x_list for val in sublist]
y_list = [val for sublist in y_list for val in sublist]
team_list = [val for sublist in team_list for val in sublist]
opposing_team_list = [val for sublist in opposing_team_list for val in sublist]
xG_list = [val for sublist in xG_list for val in sublist]
situation_list = [val for sublist in situation_list for val in sublist]
result_list = [val for sublist in result_list for val in sublist]
home_score_list = [val for sublist in home_score_list for val in sublist]
away_score_list = [val for sublist in away_score_list for val in sublist]
minute_list = [val for sublist in minute_list for val in sublist]
player_list = [val for sublist in player_list for val in sublist]
h_a_list = [val for sublist in h_a_list for val in sublist]
#Flattening nested lists

In [93]:
col_names = ['match_id','x','y','xG','minute','situation','result','team','opposing_team','h_a','player','home_score','away_score']
df = pd.DataFrame([match_id_list,x_list,y_list,xG_list,minute_list,situation_list,result_list,team_list,opposing_team_list,h_a_list,player_list,home_score_list,away_score_list],index=col_names)
df = df.T
df = df.astype({'minute':'int'})
df = df.sort_values(by=['match_id','minute'])
df = df.reset_index()

In [94]:
df.isna().sum()
#Checking for null values

index            0
match_id         0
x                0
y                0
xG               0
minute           0
situation        0
result           0
team             0
opposing_team    0
h_a              0
player           0
home_score       0
away_score       0
dtype: int64

# xG

In [95]:
# test_home_list = np.where((df['result'] == 'Goal') & (df['h_a'] == 'h') | (df['result'] == 'OwnGoal') & (df['h_a'] == 'a'), 1, 0)

In [96]:
df['xG'] = df['xG'].astype(float)
df['match_id'] = df['match_id'].astype(int)

In [97]:
xg_total = df.groupby('team')['xG'].sum()
#Count total xG for every team

In [98]:
xg_total = pd.DataFrame(data=xg_total)
# xg_total = xg_total.reset_index(level=0)

In [99]:
xg_total['xGA'] = df.groupby('opposing_team')['xG'].sum()

In [100]:
xg_total['xG_difference'] = xg_total['xG'] - xg_total['xGA']

In [101]:
xg_total['matches_played'] = df.groupby('team')['match_id'].nunique()
#Get number of matches played for each team

In [102]:
xg_total = xg_total.reset_index(level=0)

In [103]:
xg_total['avg_xG_diff'] = round(xg_total['xG_difference'] / xg_total['matches_played'], 2)
xg_total['avg_xG_for'] = round(xg_total['xG'] / xg_total['matches_played'], 2)
xg_total['avg_xGA'] = round(xg_total['xGA'] / xg_total['matches_played'], 2)

In [104]:
xg_total.isna().sum()

team              0
xG                0
xGA               0
xG_difference     0
matches_played    0
avg_xG_diff       0
avg_xG_for        0
avg_xGA           0
dtype: int64

# Non Pen xG

In [107]:
non_pen_xg = df.groupby('team')
non_pen_xg = non_pen_xg.apply(lambda x: x[x['situation'] != 'Penalty']['xG'].sum())
#Getting non pen xG and adding to df
#Not working because this groupby result does not have a name so can't go into a df

In [108]:
non_pen_xga = df.groupby('opposing_team')
non_pen_xga = non_pen_xga.apply(lambda x: x[x['situation'] != 'Penalty']['xG'].sum())

In [109]:
non_pen_xga

opposing_team
Arsenal                     7.418746
Aston Villa                10.422756
Bournemouth                14.007809
Brentford                  13.176598
Brighton                    8.687273
Chelsea                    11.063189
Crystal Palace             13.954531
Everton                    14.196854
Fulham                     21.055489
Leeds                      11.619165
Leicester                  14.431632
Liverpool                  10.873396
Manchester City             5.110035
Manchester United          11.490831
Newcastle United           11.379737
Nottingham Forest          14.328845
Southampton                14.803910
Tottenham                   9.842383
West Ham                    8.221790
Wolverhampton Wanderers    11.153440
dtype: float64

In [111]:
xg_total['non_pen_xG'] = non_pen_xg.values
xg_total['non_pen_xGA'] = non_pen_xga.values

#Now need to match for team

In [112]:
xg_total['xG_difference'] = xg_total['xG'] - xg_total['xGA']

In [113]:
xg_total['non_pen_xg_difference'] = xg_total['non_pen_xG'] - xg_total['non_pen_xGA']

In [114]:
xg_total['avg_non_pen_xG_diff'] = round(xg_total['non_pen_xg_difference'] / xg_total['matches_played'], 2)
xg_total['avg_non_pen_xG_for'] = round(xg_total['non_pen_xG'] / xg_total['matches_played'], 2)
xg_total['avg_non_pen_xGA'] = round(xg_total['non_pen_xGA'] / xg_total['matches_played'], 2)

In [115]:
xg_total = xg_total.sort_values(by='avg_xG_diff', ascending=False).reset_index(drop=True)

In [116]:
xg_total

,team,xG,xGA,xG_difference,matches_played,avg_xG_diff,avg_xG_for,avg_xGA,non_pen_xG,non_pen_xGA,non_pen_xg_difference,avg_non_pen_xG_diff,avg_non_pen_xG_for,avg_non_pen_xGA
0,Manchester City,23.390645,5.871204,17.519441,9,1.95,2.60,0.65,22.629476,5.110035,17.519441,1.95,2.51,0.57
1,Arsenal,19.776189,8.179915,11.596274,9,1.29,2.20,0.91,19.015020,7.418746,11.596274,1.29,2.11,0.82
2,Newcastle United,19.063273,12.140906,6.922367,9,0.77,2.12,1.35,18.302104,11.379737,6.922367,0.77,2.03,1.26
3,Brighton,14.556713,8.687273,5.869440,8,0.73,1.82,1.09,12.273207,8.687273,3.585934,0.45,1.53,1.09
4,Liverpool,17.153880,12.395734,4.758147,8,0.59,2.14,1.55,17.153880,10.873396,6.280484,0.79,2.14,1.36
5,Tottenham,15.155263,10.603552,4.551711,9,0.51,1.68,1.18,13.632925,9.842383,3.790542,0.42,1.51,1.09
6,West Ham,11.806591,9.744128,2.062464,9,0.23,1.31,1.08,10.284254,8.221790,2.062464,0.23,1.14,0.91
7,Manchester United,13.199327,11.490831,1.708495,8,0.21,1.65,1.44,12.438158,11.490831,0.947327,0.12,1.55,1.44
8,Chelsea,12.158590,11.063189,1.095401,8,0.14,1.52,1.38,11.397421,11.063189,0.334232,0.04,1.42,1.38
9,Brentford,13.534295,13.176598,0.357697,9,0.04,1.50,1.46,12.011957,13.176598,-1.164641,-0.13,1.33,1.46


In [ ]:
xg_total.to_excel("prem_xG_data.xlsx")